In [1]:
!pip install scipy==1.7.3
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install lightgbm==3.3.2
!pip install catboost==1.2
!pip install xgboost==1.7.6
!pip install sklearn==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.3.0 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
py

In [2]:
import scipy
import numpy
import pandas
import lightgbm
import catboost
import xgboost
import sklearn
print(scipy.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(lightgbm.__version__)
print(catboost.__version__)
print(xgboost.__version__)
print(sklearn.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.7.3
1.23.5
1.5.3
3.3.2
1.2
1.7.6
1.2.2


In [3]:
import os
import warnings
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import lightgbm as lgbm
from scipy import stats
from scipy.stats import hmean,gmean
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GroupKFold
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [4]:
version_nb = 'v4_kaggle'

In [5]:
df_meta_train = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/train.csv',parse_dates=['endTime'])
df_meta_test = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/test.csv',parse_dates=['endTime'])
df_ss = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/sample_submission.csv')
df_meta_test.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
df_meta_train.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
print(df_meta_train.shape, df_meta_test.shape, df_ss.shape)
display(df_meta_train.head(1))
display(df_meta_test.head(1))
display(df_ss.head(1))

(908922, 12) (175150, 12) (1932, 2)


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0001-12_1,B0001-12,2023-02-09 01:00:00,99.519744,29.417,0.636289,10.12688,8.640631,12.523579,32.209893,0.0,0.0


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0017-25_24,B0017-25,2023-02-20 06:00:00,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0


,ID,data_rate_t+1_trend
0,B0017-25_24,0


In [6]:
df_meta_train_features = df_meta_train[df_meta_train['fault_duration']==0].reset_index(drop=True)
df_meta_test_features = df_meta_test[df_meta_test['fault_duration']==0].reset_index(drop=True)
print(df_meta_train_features.shape,df_meta_test_features.shape)
df_train = df_meta_train[df_meta_train['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_train_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_train = df_train.merge(df1,on=['ID'],how='left')
df_train['data_rate_t+1_trend'] = np.where(df_train['data_rate']<df_train['data_rate_t-1'],1,0)


# df_train_first = df_train.groupby(['ID']).first().reset_index()
# df_train_after_first = df_train.groupby(["ID"]).apply(lambda group: group.iloc[1:]).reset_index(drop=True)
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs']:
    del df_train[col]
# df_train = pd.concat([df_train_first,df_train_after_first], ignore_index=True)

print(df_train.groupby(['ID']).first()['data_rate_t+1_trend'].value_counts(normalize=True))
print(df_train['data_rate_t+1_trend'].value_counts(normalize=True))
df_train.rename(columns={'data_rate':'TARGET_data_rate','data_rate_t+1_trend':'TARGET_data_rate_t+1_trend'},inplace=True)
df_train = df_train[['ID', 'NE ID', 'endTime',  'fault_duration','relation', 'data_rate_t-1',
                     'TARGET_data_rate', 'TARGET_data_rate_t+1_trend']].copy()
display(df_train)
df_test = df_meta_test[df_meta_test['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_test_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_test = df_test.merge(df1,on=['ID'],how='left')
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs','data_rate']:
    del df_test[col]

display(df_test)
df_meta_train_features['data'] = 'no_fault'
df_meta_test_features['data'] = 'no_fault'
df_train['data'] = 'train_fault'
df_test['data'] = 'test_fault'
df_combined = pd.concat([df_meta_train_features,df_train,df_meta_test_features,df_test]).reset_index(drop=True)

(813138, 12) (173218, 12)
0    0.508407
1    0.491593
Name: data_rate_t+1_trend, dtype: float64
1    0.526936
0    0.473064
Name: data_rate_t+1_trend, dtype: float64


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1,TARGET_data_rate,TARGET_data_rate_t+1_trend
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,23.253083,23.910174,0
1,B0001-13_1,B0001-13,2023-02-20 13:00:00,91.0,0.942467,22.537942,28.421796,0
2,B0001-15_1,B0001-15,2023-02-20 13:00:00,91.0,0.942467,5.711927,19.964336,0
3,B0001-22_1,B0001-22,2023-02-20 13:00:00,94.0,0.946876,26.605883,13.039390,1
4,B0001-23_1,B0001-23,2023-02-20 13:00:00,94.0,0.946876,24.037401,35.187848,0
...,...,...,...,...,...,...,...,...
95779,B0729-13_1,B0729-13,2023-02-28 02:00:00,209.0,0.990000,21.438768,21.961071,0
95780,B0729-15_1,B0729-15,2023-02-28 01:00:00,950.0,0.990000,4.248081,4.510619,0
95781,B0729-15_1,B0729-15,2023-02-28 02:00:00,209.0,0.990000,4.248081,6.097206,0
95782,B0729-22_1,B0729-22,2023-02-28 01:00:00,950.0,1.000000,14.851790,20.272872,0


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,1.175289
1,B0017-25_25,B0017-25,2023-02-20 21:00:00,145.0,0.654162,0.966620
2,B0017-25_26,B0017-25,2023-02-21 00:00:00,250.0,0.654162,1.561278
3,B0017-25_27,B0017-25,2023-02-21 06:00:00,1971.0,0.654162,10.963935
4,B0017-32_1,B0017-32,2023-02-14 21:00:00,3461.0,0.654162,4.471131
...,...,...,...,...,...,...
1927,B0122-43_1,B0122-43,2023-02-11 15:00:00,616.0,1.000000,16.973420
1928,B0122-43_2,B0122-43,2023-02-13 15:00:00,97.0,0.990000,8.685347
1929,B0122-45_1,B0122-45,2023-02-11 15:00:00,616.0,1.000000,3.946578
1930,B0122-45_2,B0122-45,2023-02-13 15:00:00,97.0,0.990000,2.349000


In [7]:
%%time
df_total = df_combined.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
del df_total['data_rate_t-1']
df_total_resample = df_total.set_index('endTime').groupby('NE ID').resample('H').asfreq().drop(['NE ID'], 1).reset_index()
df_total_resample = df_total_resample.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
print(df_total.shape, df_total_resample.shape)

(1084072, 15) (1093242, 15)
CPU times: user 10.9 s, sys: 1.37 s, total: 12.3 s
Wall time: 12.3 s


In [8]:
# df_total['access_success_rate_by_resource_utilition_rate'] = df_total['access_success_rate'] / df_total['resource_utilition_rate']
# df_total_resample['access_success_rate_by_resource_utilition_rate'] = df_total_resample['access_success_rate'] / df_total_resample['resource_utilition_rate']


In [9]:
# df_total = df_total[df_total['data_rate']!=0].reset_index(drop=True)
all_shits = [1,2,3] #list(np.arange(1,49)) # 
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T-{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(shift_i)
    
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T+{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(-shift_i)
        

for shift_i in tqdm(all_shits):
    df_total[f'endTime_T+{shift_i}_hours_elapsed'] = (df_total[f'endTime_T+{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    df_total[f'endTime_T-{shift_i}_hours_elapsed'] = (df_total[f'endTime_T-{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    del df_total[f'endTime_T+{shift_i}']
    del df_total[f'endTime_T-{shift_i}']
    

100%|██████████| 3/3 [00:00<00:00, 11.56it/s]


In [10]:
# df_total[f'data_rate_T-1-T-2'] = df_total.groupby(['NE ID'])['data_rate'].shift(1)- df_total.groupby(['NE ID'])['data_rate'].shift(2)
# df_total[f'data_rate_T-2-T-3'] = df_total.groupby(['NE ID'])['data_rate'].shift(2)- df_total.groupby(['NE ID'])['data_rate'].shift(3)
# df_total[f'data_rate_T-3-T-4'] = df_total.groupby(['NE ID'])['data_rate'].shift(3)- df_total.groupby(['NE ID'])['data_rate'].shift(4)


In [11]:
for shift_i in tqdm([24,48,72]):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T-{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(shift_i)

for shift_i in tqdm([24,48,72]):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T+{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(-shift_i)


100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


In [12]:
df_tot_fault_et = df_total_resample[df_total_resample['fault_duration']>0].groupby(['NE ID']).agg({'endTime':lambda x: list(x)}).add_prefix("fault_").reset_index()


In [13]:
# df_total_resample[f'data_rate_T-24-T-25'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(24)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(25)
# df_total_resample[f'data_rate_T-48-T-49'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(48)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(49)
# df_total_resample[f'data_rate_T-72-T-73'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(72)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(73)

In [14]:
df_total['month'] = df_total['endTime'].dt.month
df_total['day'] = df_total['endTime'].dt.day
df_total['hour'] = df_total['endTime'].dt.hour
df_total['weekday_number'] = df_total['endTime'].dt.weekday
df_total['hour_sin'] = np.sin(2 * np.pi * df_total['hour'] / 24)
df_total['hour_cos'] = np.cos(2 * np.pi * df_total['hour'] / 24)
# df_total['weekday_sin'] = np.sin(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['weekday_cos'] = np.cos(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['day_sin'] = np.sin(2 * np.pi * df_total['day'] / 31)
# df_total['day_cos'] = np.cos(2 * np.pi * df_total['day'] / 31)

# df_total = df_total.groupby(['ID']).first().reset_index()


In [15]:
# df_mean_drates = df_total.groupby(['NE ID','hour']).agg({'data_rate':['mean','std','min','max']})
# df_mean_drates.columns = [i[0]+"_"+i[1] for i in df_mean_drates]
# df_mean_drates = df_mean_drates.reset_index()
# df_total = df_total.merge(df_mean_drates,on=['NE ID','hour'],how='left')

In [16]:
# category_cols = ['month','day','hour','weekday_number']

In [17]:
df_total['last_10_data_rates'] = np.vstack([np.array(df_total.groupby(['NE ID'])['data_rate'].shift(i).fillna(-1)) for i in range(1,11)]).T.tolist()


In [18]:
df_train = df_total[df_total['data']=='train_fault'].groupby(['ID']).first().reset_index()
df_train.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data','TARGET_data_rate'],
              inplace=True)
df_train = df_train.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_train = df_train.merge(df_tot_fault_et,on='NE ID',how='left')
# df_train['fault_endTime'] = df_train.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_train['fault_endTime'] = (df_train['endTime']-df_train['fault_endTime']).dt.total_seconds() / 3600

print(df_train.shape)
display(df_train.head(1))

(7256, 127)


,ID,NE ID,endTime,fault_duration,relation,TARGET_data_rate_t+1_trend,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,0.0,100.0,29.382,0.763943,10.336147,8.241281,11.089316,23.253083,0.0,0.0,100.0,45.419,0.811247,10.261306,8.565688,12.344837,21.86461,0.0,0.0,100.0,45.113,0.75943,10.249399,8.600942,12.360695,24.524838,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,-2.0,NaN,-3.0,2,20,13,0,-0.258819,-0.965926,"[23.25308286587564, 21.864609567544047, 24.524...",100.0,25.604,0.891304,10.965842,8.320807,11.496593,17.156717,0.0,0.0,100.0,37.071,0.862605,11.659417,8.081238,10.35328,17.371207,0.0,0.0,99.951714,31.598,0.834253,10.487901,8.34011,9.946214,22.470461,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_test = df_total[df_total['data']=='test_fault'].reset_index(drop=True)
df_test.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data',
                      'TARGET_data_rate','TARGET_data_rate_t+1_trend'],
             inplace=True)
df_test = df_test.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_test = df_test.merge(df_tot_fault_et,on='NE ID',how='left')
# df_test['fault_endTime'] = df_test.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_test['fault_endTime'] = (df_test['endTime']-df_test['fault_endTime']).dt.total_seconds() / 3600


print(df_test.shape)
display(df_test.head(1))

(1932, 126)


,ID,NE ID,endTime,fault_duration,relation,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,99.357688,84.004,2.923368,14.209819,5.582824,5.667775,1.175289,0.0,0.0,99.927484,24.264,3.145402,10.015796,6.141206,6.544645,6.777114,0.0,0.0,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0,99.264875,93.086,2.732594,16.947693,5.475508,4.719393,0.52254,0.0,0.0,99.642289,92.242,2.877206,15.083843,5.628569,5.051611,0.96662,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,0.654162,10.0,-1.0,11.0,-2.0,12.0,-3.0,2,20,9,0,0.707107,-0.707107,"[1.17528900605546, 6.777114273500261, 6.815788...",99.814929,55.07,3.171086,12.769518,5.834403,6.893467,3.292267,0.0,0.0,99.754199,78.256,2.889007,13.597193,5.825114,6.012004,2.046892,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0,0.654162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
remove_non_unique_cols = []
for col in df_train:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_train[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_train[col].nunique())

for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_test[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_test[col].nunique())
remove_non_unique_cols

fault_duration_T-1 : 1
relation_T-1 : 1
fault_duration_T+1 : 1
relation_T+1 : 1


['fault_duration_T-1', 'relation_T-1', 'fault_duration_T+1', 'relation_T+1']

In [21]:
feature_set = [col for col in df_test.columns if col not in ['ID','NE ID','endTime','last_10_data_rates']]
print(len(feature_set))
feature_set = [col for col in feature_set if col not in remove_non_unique_cols]
print(len(feature_set))
target_variable = 'TARGET_data_rate_t+1_trend'
id_variable = 'ID'

122
118


In [22]:
for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend']:
        if df_test[col].isnull().sum()/len(df_test)>=0.8:
            print(col,":",df_test[col].isnull().sum()/len(df_test))

In [23]:
train_x = df_train.reset_index(drop=True)
train_y = train_x[target_variable]
test_x = df_test.reset_index(drop=True)

del train_x[target_variable]
test_id = test_x[id_variable]
# train_x = train_x[feature_set].copy()
# test_x = test_x[feature_set].copy()

print(train_x.shape, train_y.shape, test_x.shape)

(7256, 126) (7256,) (1932, 126)


In [24]:
def evaluate_F1_lgb(truth, predictions):  
    predictions = np.where(predictions < f1_threshold, 0, 1)
    f1 = f1_score(truth, predictions)
    return ('F1 Score', f1, True)

def evaluate_F1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = f1_score(y_true, np.round(y_pred))
    return 'f1_err', -err

NFOLDS = 10
sc_pos_wt = 1.5
f1_threshold=0.5
seeds_rs = [42] #,1818,181812,1995,1212

In [25]:
#0.7293
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_lgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = LGBMClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                               n_jobs=-1,lambda_l1=1,feature_fraction=0.45,
                               scale_pos_weight=sc_pos_wt,
#                                max_depth=8,
#                                linear_tree=True,linear_lambda=0.1
                              )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
#                   categorical_feature=['endTime'],
                  eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_lgb = pd.concat([df_train_error_analysis_lgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_lgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"lgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"lgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_lgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.7439613526570048


● Fold : 2
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.73094688221709


● Fold : 3
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.7291428571428571


● Fold : 4
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.587532,0.573935,0.630036,0.606252,0.657784,0.695109,0.667437,0.633670,0.714693,0.585352,B0017-25_24,0.631955
1,0.231523,0.383633,0.468857,0.254684,0.350040,0.224774,0.236996,0.162006,0.174898,0.299898,B0017-25_25,0.251667
2,0.167446,0.341879,0.235279,0.239052,0.253468,0.166000,0.319152,0.165843,0.115748,0.213120,B0017-25_26,0.200825
3,0.852125,0.666311,0.731601,0.774812,0.745461,0.811305,0.709236,0.800230,0.781014,0.782835,B0017-25_27,0.762005
4,0.830704,0.735359,0.788456,0.784555,0.810137,0.883988,0.797309,0.851709,0.917133,0.811409,B0017-32_1,0.818110


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,403.4
1,data_rate_T-72,161.8
2,data_rate_T+1,118.1
3,data_rate_T-24,112.1
4,data_rate_T-48,102.6
...,...,...
113,endTime_T-3_hours_elapsed,1.0
114,endTime_T+1_hours_elapsed,0.9
115,endTime_T-2_hours_elapsed,0.6
116,endTime_T-1_hours_elapsed,0.5


,feature,importance
0,data_rate_T-1,403.4
1,data_rate_T-72,161.8
2,data_rate_T+1,118.1
3,data_rate_T-24,112.1
4,data_rate_T-48,102.6
5,data_rate_T-3,101.4
6,mcs_T-1,87.6
7,data_rate_T-2,85.4
8,data_rate_T+2,82.3
9,fault_duration,79.9


In [26]:
#0.7228
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_cb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = CatBoostClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                                   eval_metric='F1:use_weights=False',
                                   scale_pos_weight=sc_pos_wt,
                                   use_best_model=True,
                                   max_depth=8
                                  )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  use_best_model=True,
    #               cat_features=category_cols
    #               eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    

    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_cb = pd.concat([df_train_error_analysis_cb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_cb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"cb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"cb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_cb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.744131455399061


● Fold : 2
FOLD F1 Score: 0.7312072892938498


● Fold : 3
FOLD F1 Score: 0.7254389241688457


● Fold : 4
FOLD F1 Score: 0.7246376811594202


● Fold : 5
FOLD F1 Score: 0.7260058440098899


● Fold : 6
FOLD F1 Score: 0.7260145876192259


● Fold : 7
FOLD F1 Score: 0.7243467176545572


● Fold : 8
FOLD F1 Score: 0.7232192933258553


● Fold : 9
FOLD F1 Score: 0.7219947897283215


● Fold : 10
FOLD F1 Score: 0.7228156977397953


Final F1 Score: 0.7228156977397953
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.711764,0.601021,0.606487,0.585884,0.667871,0.709458,0.565981,0.671044,0.604489,0.695725,B0017-25_24,0.637756
1,0.270192,0.407265,0.467367,0.474175,0.312965,0.404475,0.486068,0.331568,0.455564,0.218439,B0017-25_25,0.358180
2,0.161770,0.266217,0.317720,0.290013,0.183883,0.240558,0.318284,0.185372,0.287879,0.180386,B0017-25_26,0.228925
3,0.862357,0.766286,0.640490,0.701651,0.751546,0.771480,0.641170,0.792252,0.690183,0.775903,B0017-25_27,0.733272
4,0.754718,0.760803,0.613932,0.665191,0.832713,0.760883,0.641164,0.690999,0.702004,0.822322,B0017-32_1,0.717662


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,19.048753
1,hour,4.169915
2,mcs_T-1,2.976458
3,data_rate_T+1,2.864438
4,data_rate_T-72,2.634389
...,...,...
113,endTime_T+3_hours_elapsed,0.046828
114,endTime_T-2_hours_elapsed,0.045287
115,endTime_T+1_hours_elapsed,0.039125
116,endTime_T-1_hours_elapsed,0.027085


,feature,importance
0,data_rate_T-1,19.048753
1,hour,4.169915
2,mcs_T-1,2.976458
3,data_rate_T+1,2.864438
4,data_rate_T-72,2.634389
5,hour_cos,2.367850
6,data_rate_T-48,2.234078
7,TA_T-1,2.181382
8,data_rate_T+2,2.086460
9,data_rate_T-24,1.879623


In [27]:
#0.7184
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_hgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = HistGradientBoostingClassifier(random_state=rs_,verbose=0,
                                               scoring = 'recall',
                                              early_stopping = True,
                                               max_iter = 10000,
                                              learning_rate = 0.03,
                                              n_iter_no_change = 100,
                                               class_weight={1:sc_pos_wt},
                                               )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids])
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_hgb = pd.concat([df_train_error_analysis_hgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
#     fold_importance_df["importance"] = model.feat
#     fold_importance_df["fold"] = j + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_hgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"hgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"hgb_{version_nb}.csv", index = False)
display(df_submission.head())

# all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
# all_features.reset_index(inplace=True)
# important_features = list(all_features[all_features['importance']!=0]['feature'])
# display(all_features)
# display(all_features.head(25))

final_prediction_hgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.7405660377358491


● Fold : 2
FOLD F1 Score: 0.7278037383177571


● Fold : 3
FOLD F1 Score: 0.7240841777084959


● Fold : 4
FOLD F1 Score: 0.7196702002355713


● Fold : 5
FOLD F1 Score: 0.7195842192298606


● Fold : 6
FOLD F1 Score: 0.7233124018838305


● Fold : 7
FOLD F1 Score: 0.7213611859838275


● Fold : 8
FOLD F1 Score: 0.7197047970479705


● Fold : 9
FOLD F1 Score: 0.7180559184740005


● Fold : 10
FOLD F1 Score: 0.7184557438794728


Final F1 Score: 0.7184557438794728
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.571245,0.537129,0.727261,0.538256,0.695793,0.730116,0.658118,0.633415,0.568114,0.584654,B0017-25_24,0.616489
1,0.388289,0.273384,0.410010,0.234379,0.357188,0.274236,0.178054,0.329629,0.243732,0.339625,B0017-25_25,0.285044
2,0.271201,0.260808,0.220356,0.270553,0.175407,0.252238,0.225323,0.266760,0.190115,0.304338,B0017-25_26,0.237226
3,0.789734,0.787034,0.763241,0.834388,0.754907,0.757299,0.791133,0.764331,0.747900,0.786148,B0017-25_27,0.776873
4,0.764659,0.851587,0.877043,0.823052,0.861333,0.847964,0.870292,0.832842,0.878085,0.783704,B0017-32_1,0.837373


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


In [28]:
# train_x[category_cols] = train_x[category_cols].astype("category")
# test_x[category_cols] = test_x[category_cols].astype("category")
#  enable_categorical=True,

In [29]:
# 0.7282
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_xgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = XGBClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                              n_jobs=-1,scale_pos_weight=sc_pos_wt,colsample_bytree=0.45,
                              reg_lambda=1,reg_alpha=0,max_depth=7,
                             disable_default_eval_metric=1,tree_method='exact')
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  eval_metric=evaluate_F1_xgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_xgb = pd.concat([df_train_error_analysis_xgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_xgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"xgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"xgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_xgb = final_prediction[['ID','data_rate_t+1_trend']].copy()



● Fold : 1
FOLD F1 Score: 0.7408256880733946


● Fold : 2
FOLD F1 Score: 0.7329902801600914


● Fold : 3
FOLD F1 Score: 0.7299546142208775


● Fold : 4
FOLD F1 Score: 0.7256637168141593


● Fold : 5
FOLD F1 Score: 0.7277726856095325


● Fold : 6
FOLD F1 Score: 0.7327304213969597


● Fold : 7
FOLD F1 Score: 0.730169157497126


● Fold : 8
FOLD F1 Score: 0.7298001737619461


● Fold : 9
FOLD F1 Score: 0.7272263982672952


● Fold : 10
FOLD F1 Score: 0.7282558674643351


Final F1 Score: 0.7282558674643351
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.455124,0.730460,0.658256,0.561869,0.673780,0.691353,0.655958,0.730138,0.554765,0.618605,B0017-25_24,0.620773
1,0.179200,0.177466,0.320230,0.149121,0.241084,0.136465,0.311474,0.105387,0.372238,0.126234,B0017-25_25,0.179124
2,0.248869,0.195111,0.203206,0.141711,0.210145,0.111276,0.302184,0.107436,0.334938,0.114321,B0017-25_26,0.168872
3,0.758639,0.735105,0.694270,0.743621,0.745263,0.849142,0.647122,0.788243,0.616221,0.807874,B0017-25_27,0.732190
4,0.742045,0.814640,0.759061,0.756189,0.786829,0.889539,0.694980,0.828342,0.620011,0.818641,B0017-32_1,0.763905


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,0.025750
1,hour,0.023582
2,data_rate_T+1,0.019396
3,cqi_T-1,0.019277
4,TA_T-1,0.015846
...,...,...
113,access_success_rate_T+48,0.006211
114,endTime_T+1_hours_elapsed,0.006195
115,mcs_T+72,0.006177
116,fault_duration_T-24,0.005848


,feature,importance
0,data_rate_T-1,0.025750
1,hour,0.023582
2,data_rate_T+1,0.019396
3,cqi_T-1,0.019277
4,TA_T-1,0.015846
5,resource_utilition_rate_T-1,0.015195
6,data_rate_T+2,0.013972
7,data_rate_T-48,0.013875
8,data_rate_T-72,0.013645
9,hour_cos,0.012744


In [30]:
for df in [final_prediction_lgb,final_prediction_cb,final_prediction_hgb,final_prediction_xgb]:
    print(df['data_rate_t+1_trend'].min(),df['data_rate_t+1_trend'].max())

0.048700636504055456 0.8795129244765755
0.0529197467982966 0.9005857165428655
0.031199330830530544 0.8968286270953256
0.0440802926952185 0.8519575861496931


In [31]:
#1.5 - 0.7248
df_local_cv = df_train_error_analysis_lgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_LGB").join(df_train_error_analysis_xgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_XGB")).join(df_train_error_analysis_cb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_CB")).join(df_train_error_analysis_hgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_HGB"))
# vote = df_local_cv['PREDICTED_TARGET_LGB']*0.4+df_local_cv['PREDICTED_TARGET_XGB']*0.4+df_local_cv['PREDICTED_TARGET_HGB']*0.1+df_local_cv['PREDICTED_TARGET_CB']*0.1
vote = df_local_cv.apply(lambda x: hmean(x), axis=1)
df_local_cv['data_rate_t+1_trend'] = vote
df_local_cv['data_rate_t+1_trend'] = list(np.where(df_local_cv['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_local_cv = df_local_cv.reset_index()
f1_score(df_local_cv['ACTUAL_TARGET'],df_local_cv['data_rate_t+1_trend'])

0.7248259860788864

In [32]:
df_ensemble = final_prediction_lgb.set_index("ID").add_suffix("_LGB").join(final_prediction_cb.set_index("ID").add_suffix("_CB")).join(final_prediction_hgb.set_index("ID").add_suffix("_HGB")).join(final_prediction_xgb.set_index("ID").add_suffix("_XGB"))
display(df_ensemble.corr())
vote = df_ensemble.apply(lambda x: hmean(x), axis=1)
# vote = df_ensemble['data_rate_t+1_trend_LGB']*0.4+df_ensemble['data_rate_t+1_trend_XGB']*0.4+df_ensemble['data_rate_t+1_trend_HGB']*0.1+df_ensemble['data_rate_t+1_trend_CB']*0.1
df_ensemble['data_rate_t+1_trend'] = vote
df_ensemble = df_ensemble[['data_rate_t+1_trend']].copy()
df_ensemble_raw = df_ensemble.reset_index()
df_ensemble_raw.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}_raw.csv", index = False)
df_ensemble['data_rate_t+1_trend'] = list(np.where(df_ensemble['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_ensemble = df_ensemble.reset_index()
df_ensemble.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}.csv", index = False)


,data_rate_t+1_trend_LGB,data_rate_t+1_trend_CB,data_rate_t+1_trend_HGB,data_rate_t+1_trend_XGB
data_rate_t+1_trend_LGB,1.000000,0.937557,0.975219,0.987149
data_rate_t+1_trend_CB,0.937557,1.000000,0.919405,0.945602
data_rate_t+1_trend_HGB,0.975219,0.919405,1.000000,0.966960
data_rate_t+1_trend_XGB,0.987149,0.945602,0.966960,1.000000


In [33]:
df_ensemble['data_rate_t+1_trend'].value_counts(normalize=True)

1    0.713251
0    0.286749
Name: data_rate_t+1_trend, dtype: float64